# Data Preparation

In [1]:
import pandas as pd
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)

## Drivers from FRED

- Add growth rates

In [2]:
import pandas as pd
df_derived = pd.read_csv('./data/raw_FRED.csv')

growth_vars = [  # Growth rates (QoQ, YoY)
    'M1', 'M2', 'M3', 'CURRENCY', 'MONETARY_BASE',
    'GDP', 'REAL_GDP', 'PERSONAL_INCOME', 'DISPOSABLE_INCOME', 'DISPOSABLE_INCOME_NOMINAL',
    'PCE', 'REAL_PCE', 'PERSONAL_SAVINGS',
    'CPI', 'CORE_CPI', 'PCE_INFLATION', 'CORE_PCE', 'PPI',
    'NONFARM_PAYROLL', 'LABOR_FORCE', 'AVERAGE_HOURLY_EARNINGS',
    'INDUSTRIAL_PRODUCTION', 'RETAIL_SALES', 'RETAIL_SALES_EXCL_AUTO',
    'HOUSE_PRICE_INDEX', 'HOUSING_STARTS', 'NEW_HOME_SALES', 'EXISTING_HOME_SALES',
    'COMMERCIAL_LOANS', 'CONSUMER_CREDIT', 'MORTGAGE_DEBT', 'BANK_CREDIT',
    'ASSETS_COMMERCIAL_BANKS', 'MANUFACTURING_NEW_ORDERS', 'BUSINESS_INVENTORIES',
    'SP500', 'WILSHIRE_5000', 'OIL_PRICE', 'GOLD_PRICE',
    'FEDERAL_DEBT', 'GOVERNMENT_SPENDING'
]
for var in growth_vars:
    if var in df_derived.columns:
        df_derived[f'{var}_QOQ'] = df_derived[var].pct_change(fill_method=None) * 100
        df_derived[f'{var}_YOY'] = df_derived[var].pct_change(4, fill_method=None) * 100

- Add Interest rate changes

In [3]:
# Interest rate changes (level changes)
rate_vars = [
    'FED_FUNDS', 'FED_FUNDS_TARGET',
    'TREASURY_1M', 'TREASURY_3M', 'TREASURY_6M', 'TREASURY_1Y',
    'TREASURY_2Y', 'TREASURY_3Y', 'TREASURY_5Y', 'TREASURY_7Y',
    'TREASURY_10Y', 'TREASURY_20Y', 'TREASURY_30Y',
    'AAA_CORPORATE', 'BBB_CORPORATE', 'HIGH_YIELD',
    'MORTGAGE_30Y', 'MORTGAGE_15Y', 'NET_INTEREST_MARGIN']
for var in rate_vars:
    if var in df_derived.columns:
        df_derived[f'{var}_CHG'] = df_derived[var].diff()

- Add pct_vars change

In [4]:
pct_vars = [
    'UNEMPLOYMENT', 'LABOR_FORCE_PARTICIPATION', 'EMPLOYMENT_POPULATION_RATIO',
    'SAVING_RATE', 'CAPACITY_UTILIZATION', 'HOME_OWNERSHIP_RATE',
    'CREDIT_CARD_DELINQUENCY', 'AUTO_LOAN_DELINQUENCY']
for var in pct_vars:
    if var in df_derived.columns:
        df_derived[f'{var}_CHG'] = df_derived[var].diff()

- Add spreads

In [5]:
# yield curve spread
yield_spreads = [
    ('TREASURY_10Y', 'TREASURY_2Y', '10Y2Y'),
    ('TREASURY_10Y', 'TREASURY_3M', '10Y3M'),
    ('TREASURY_10Y', 'TREASURY_1Y', '10Y1Y'),
    ('TREASURY_5Y', 'TREASURY_2Y', '5Y2Y'),
    ('TREASURY_30Y', 'TREASURY_10Y', '30Y10Y'),
    ('TREASURY_2Y', 'TREASURY_3M', '2Y3M')]
for long_rate, short_rate, spread_name in yield_spreads:
    if long_rate in df_derived.columns and short_rate in df_derived.columns:
        df_derived[f'YIELD_CURVE_{spread_name}'] = df_derived[long_rate] - df_derived[short_rate]
        df_derived[f'YIELD_CURVE_{spread_name}_CHG'] = df_derived[f'YIELD_CURVE_{spread_name}'].diff()

#credit spreads
credit_spreads = [
    ('BBB_CORPORATE', 'TREASURY_10Y', 'BBB_10Y'),
    ('AAA_CORPORATE', 'TREASURY_10Y', 'AAA_10Y'),
    ('HIGH_YIELD', 'TREASURY_10Y', 'HY_10Y'),
    ('BBB_CORPORATE', 'AAA_CORPORATE', 'BBB_AAA')]
for corp_rate, base_rate, spread_name in credit_spreads:
    if corp_rate in df_derived.columns and base_rate in df_derived.columns:
        df_derived[f'CREDIT_SPREAD_{spread_name}'] = df_derived[corp_rate] - df_derived[base_rate]
        df_derived[f'CREDIT_SPREAD_{spread_name}_CHG'] = df_derived[f'CREDIT_SPREAD_{spread_name}'].diff()

# Mortgage spread
if 'MORTGAGE_30Y' in df_derived.columns and 'TREASURY_10Y' in df_derived.columns:
    df_derived['MORTGAGE_SPREAD'] = df_derived['MORTGAGE_30Y'] - df_derived['TREASURY_10Y']
    df_derived['MORTGAGE_SPREAD_CHG'] = df_derived['MORTGAGE_SPREAD'].diff()

# Real vs Nominal spreads
if 'FED_FUNDS' in df_derived.columns and 'CPI_YOY' in df_derived.columns:
    df_derived['REAL_FED_FUNDS'] = df_derived['FED_FUNDS'] - df_derived['CPI_YOY']

if 'TREASURY_10Y' in df_derived.columns and 'CPI_YOY' in df_derived.columns:
    df_derived['REAL_TREASURY_10Y'] = df_derived['TREASURY_10Y'] - df_derived['CPI_YOY']

- Ratios and derived metrics

In [6]:
if 'PERSONAL_SAVINGS' in df_derived.columns and 'DISPOSABLE_INCOME_NOMINAL' in df_derived.columns:
    df_derived['SAVINGS_TO_INCOME_RATIO'] = (df_derived['PERSONAL_SAVINGS'] / df_derived['DISPOSABLE_INCOME_NOMINAL']) * 100
    
if 'CONSUMER_CREDIT' in df_derived.columns and 'DISPOSABLE_INCOME_NOMINAL' in df_derived.columns:
    df_derived['CREDIT_TO_INCOME_RATIO'] = (df_derived['CONSUMER_CREDIT'] / df_derived['DISPOSABLE_INCOME_NOMINAL']) * 100
    
if 'FEDERAL_DEBT' in df_derived.columns and 'GDP' in df_derived.columns:
    df_derived['DEBT_TO_GDP_RATIO'] = (df_derived['FEDERAL_DEBT'] / df_derived['GDP']) * 100

- Index change

In [7]:
index_vars = ['CONSUMER_SENTIMENT', 'CONSUMER_CONFIDENCE', 'VIX', 'ECONOMIC_POLICY_UNCERTAINTY']
for var in index_vars:
    if var in df_derived.columns:
        df_derived[f'{var}_CHG'] = df_derived[var].diff()
        df_derived[f'{var}_PCT_CHG'] = df_derived[var].pct_change() * 100

- Lagged variables

In [8]:
new_columns = []

# Extended lagged variables with multiple time periods
lag_periods = [1, 2, 3, 4, 8, 12]  # Quarter lags (1Q, 2Q, 3Q, 1Y, 2Y, 3Y)

# Core economic indicators - more comprehensive lags
core_lag_vars = [
    'M1', 'M2', 'M3',
    'GDP', 'REAL_GDP', 
    'PERSONAL_INCOME', 'DISPOSABLE_INCOME',
    'UNEMPLOYMENT', 'NONFARM_PAYROLL',
    'FED_FUNDS', 'TREASURY_10Y', 'TREASURY_2Y',
    'CPI', 'CORE_CPI', 'PCE_INFLATION'
]

for var in core_lag_vars:
    if var in df_derived.columns:
        for lag in lag_periods:
            new_columns.append(
                df_derived[var].shift(lag).rename(f'{var}_LAG{lag}')
            )

# Growth rates - lagged (QoQ and YoY)
growth_lag_vars = [
    'M2_QOQ', 'M2_YOY',
    'REAL_GDP_QOQ', 'REAL_GDP_YOY',
    'CPI_QOQ', 'CPI_YOY',
    'NONFARM_PAYROLL_QOQ', 'NONFARM_PAYROLL_YOY'
]

for var in growth_lag_vars:
    if var in df_derived.columns:
        for lag in [1, 2, 4]:  # 1Q, 2Q, 1Y back
            new_columns.append(
                df_derived[var].shift(lag).rename(f'{var}_LAG{lag}')
            )

# Interest rate changes - lagged
rate_change_vars = [
    'FED_FUNDS_CHG', 
    'TREASURY_10Y_CHG', 'TREASURY_2Y_CHG',
    'YIELD_CURVE_10Y2Y', 'YIELD_CURVE_10Y3M'
]

for var in rate_change_vars:
    if var in df_derived.columns:
        for lag in [1, 2, 4]:
            new_columns.append(
                df_derived[var].shift(lag).rename(f'{var}_LAG{lag}')
            )

# Rolling averages (smoothed features)
rolling_windows = [4, 8, 12]  # 1Y, 2Y, 3Y moving averages

rolling_vars = [
    'M2', 'REAL_GDP', 'UNEMPLOYMENT', 'FED_FUNDS', 
    'CPI', 'NONFARM_PAYROLL', 'RETAIL_SALES'
]

for var in rolling_vars:
    if var in df_derived.columns:
        for window in rolling_windows:
            new_columns.append(
                df_derived[var].rolling(window=window).mean().rename(f'{var}_MA{window}')
            )

# Rate of change features (momentum)
roc_vars = [
    'M2', 'REAL_GDP', 'UNEMPLOYMENT', 'FED_FUNDS', 'CPI'
]

for var in roc_vars:
    if var in df_derived.columns:
        # 1Q momentum
        new_columns.append(
            (df_derived[var] - df_derived[var].shift(1)).rename(f'{var}_ROC1')
        )
        # 1Y momentum
        new_columns.append(
            (df_derived[var] - df_derived[var].shift(4)).rename(f'{var}_ROC4')
        )

# Real interest rate lags
for lag in [1, 2, 4]:
    fed_lag = f'FED_FUNDS_LAG{lag}'
    cpi_lag = f'CPI_LAG{lag}'
    treas_lag = f'TREASURY_10Y_LAG{lag}'
    

df_derived = pd.concat([df_derived] + new_columns, axis=1)
print(df_derived.shape)

(104, 256)


## Target

- Calculate average quarter balance and growth

In [9]:
df_deposits = pd.read_csv('data/raw_FDIC_deposits.csv')
target_cols = [col for col in df_deposits.columns if col !='timekey']

for col in target_cols:
    avg_col = f'{col}_Avg'
    df_deposits[avg_col] = (df_deposits[col] + df_deposits[col].shift(1)) / 2
    df_deposits[f'{col}_Avg_growth_ratio'] = df_deposits[avg_col].shift(-1) / df_deposits[avg_col]
    df_deposits[f'{col}_Avg_growth_ratio_yoy'] = df_deposits[avg_col].shift(-4) / df_deposits[avg_col]
df_deposits.columns

Index(['timekey', 'Domestic office deposits', 'Interest-bearing deposits', 'Noninterest-bearing deposits', 'Time deposits', 'Domestic office deposits_Avg', 'Domestic office deposits_Avg_growth_ratio', 'Domestic office deposits_Avg_growth_ratio_yoy', 'Interest-bearing deposits_Avg', 'Interest-bearing deposits_Avg_growth_ratio', 'Interest-bearing deposits_Avg_growth_ratio_yoy', 'Noninterest-bearing deposits_Avg', 'Noninterest-bearing deposits_Avg_growth_ratio', 'Noninterest-bearing deposits_Avg_growth_ratio_yoy', 'Time deposits_Avg', 'Time deposits_Avg_growth_ratio', 'Time deposits_Avg_growth_ratio_yoy'], dtype='object')

## Full Modeling Data

In [10]:
df_deposits = df_deposits[df_deposits['timekey'] >= 96]
df_modeling = df_deposits.merge(df_derived, on='timekey', how='left')
print(f'{df_modeling.shape[0]} samples, {df_derived.shape[1]} drivers.')
df_modeling.to_csv('data/full_modeling.csv',index=False)

86 samples, 256 drivers.
